# Azure AI Agent service - Function calling

<img src="https://learn.microsoft.com/en-us/azure/ai-services/agents/media/agent-service-the-glue.png" width=800>

> https://learn.microsoft.com/en-us/azure/ai-services/agents/

In [1]:
#%pip install azure-ai-projects

In [2]:
import json
import os
import requests
import sys
import time
import yfinance as yf

from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool, ToolSet, CodeInterpreterTool
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
from openai import AzureOpenAI
from typing import Any, Callable, Set, Dict, List, Optional

from user_functions import user_functions, azuremaps_weather

In [3]:
load_dotenv("azure.env")

True

In [4]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## 1. Agent definition - Weather forecasts

### Project client

In [5]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

In [6]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("PROJECT_CONNECTION_STRING"),
)

### Testing the Azure Maps Weather function

We will the Azure Maps service to provide weather information.

In [7]:
weather = azuremaps_weather("Seattle")
json.loads(weather)

{'weather_data': {'results': [{'dateTime': '2025-05-20T05:52:00-07:00',
    'phrase': 'Partly sunny',
    'iconCode': 3,
    'hasPrecipitation': False,
    'isDayTime': True,
    'temperature': {'value': 10.0, 'unit': 'C', 'unitType': 17},
    'realFeelTemperature': {'value': 10.5, 'unit': 'C', 'unitType': 17},
    'realFeelTemperatureShade': {'value': 10.5, 'unit': 'C', 'unitType': 17},
    'relativeHumidity': 90,
    'dewPoint': {'value': 8.4, 'unit': 'C', 'unitType': 17},
    'wind': {'direction': {'degrees': 180.0, 'localizedDescription': 'S'},
     'speed': {'value': 4.7, 'unit': 'km/h', 'unitType': 7}},
    'windGust': {'speed': {'value': 13.4, 'unit': 'km/h', 'unitType': 7}},
    'uvIndex': 0,
    'uvIndexPhrase': 'Low',
    'visibility': {'value': 9.7, 'unit': 'km', 'unitType': 6},
    'obstructionsToVisibility': '',
    'cloudCover': 35,
    'ceiling': {'value': 12192.0, 'unit': 'm', 'unitType': 5},
    'pressure': {'value': 1020.7, 'unit': 'mb', 'unitType': 14},
    'pressure

### Agent

In [8]:
model = "gpt-4o-mini"
name = "gpt4o-weather-agent"
instructions = "You are a weather bot. Use the provided functions to help answer questions."

# user functions to use
user_functions: Set[Callable[..., Any]] = {
    azuremaps_weather,
}

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)

# We need to define this option to the project (not in the documentation ?)
project_client.agents.enable_auto_function_calls(toolset=toolset)

# Agent creation
agent = project_client.agents.create_agent(
    model=model,
    name=name,
    instructions=instructions,
    toolset=toolset,
)

print(f"Created agent with ID = {agent.id}")

Created agent with ID = asst_6RjTsIQvw7hACHCXHdL7mHs6


### Tests

In [9]:
prompt = "Hello, generate a full report for the weather in Paris today"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_ztYX1P79BvXCR5i9dtG03Rzc
Created message ID = msg_F2mtYsKozOKzrKQrXxOhrqUH


In [10]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Created run, run ID: {run.id}")

Created run, run ID: run_JSlG2ZHNs77bLzbE3SEJGw9x


In [11]:
start   = run.started_at       
end     = run.completed_at
elapsed = end - start

iso_fmt = "%Y-%m-%d %H:%M:%S%z"
print(f"Start   : {start.strftime(iso_fmt)}")
print(f"End     : {end.strftime(iso_fmt)}")
print(f"Elapsed : {elapsed}  "
      f"({elapsed.total_seconds():.2f} seconds)")

Start   : 2025-05-20 13:05:05+0000
End     : 2025-05-20 13:05:10+0000
Elapsed : 0:00:05  (5.00 seconds)


In [12]:
run.usage

{'prompt_tokens': 1494, 'completion_tokens': 394, 'total_tokens': 1888, 'prompt_token_details': {'cached_tokens': 0}}

In [13]:
messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


### Weather Report for Paris

**Date:** May 20, 2025  
**Time:** 14:56 (GMT+2)

#### General Weather Conditions
- **Condition:** Sunny
- **Icon:** ☀️
- **Day/Night:** Daytime

#### Temperature Information
- **Current Temperature:** 24.0°C
- **Real Feel Temperature:** 25.7°C
- **Apparent Temperature:** 22.8°C
- **Relative Humidity:** 31%
- **Dew Point:** 5.8°C
- **Wind Chill Temperature:** 23.9°C

#### Wind Conditions
- **Wind Speed:** 17.1 km/h
- **Wind Direction:** NNE (23 degrees)
- **Wind Gust Speed:** 35.4 km/h

#### Visibility
- **Visibility:** 24.1 km

#### Precipitation
- **Precipitation in the past hour:** 0.0 mm
- **Precipitation in the past 24 hours:** 0.0 mm

#### Cloud Cover
- **Cloud Cover:** 0% (Clear skies)
- **Ceiling Height:** 12,192 m

#### Atmospheric Pressure
- **Pressure:** 1017.0 mb
- **Pressure Tendency:** Falling

#### Temperature Summary (Past 24 Hours)
- **Minimum Temperature:** 11.3 °C
- **Maximum Temperature:** 24.0 °C

#### UV Index
- **UV Index:** 6 (High

### Other examples

In [14]:
prompt = "Hello, generate a full report for the weather in London today. Print the results in English and French."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_gpqSHIDokvOPA0hxuNETWppW
Created message ID = msg_xXeEJJ2CRv5elCaxTjRT54Cx


In [15]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


### Weather Report for London Today

**Date and Time:** 20th May 2025, 14:01 UTC+1

**Conditions:**
- **Weather Phrase:** Sunny
- **Precipitation:** No
- **Cloud Cover:** 10%
- **Visibility:** 59.5 km
- **UV Index:** Moderate (5)

**Temperature:**
- **Current Temperature:** 21.3 °C
- **Real Feel Temperature:** 24.3 °C
- **Wind Chill Temperature:** 21.1 °C
- **Past 24 Hours Max Temperature:** 21.3 °C
- **Past 24 Hours Min Temperature:** 9.1 °C

**Humidity and Dew Point:**
- **Relative Humidity:** 39%
- **Dew Point:** 6.7 °C

**Wind:**
- **Direction:** North-Northeast (23°)
- **Speed:** 10 km/h
- **Wind Gusts:** 25.6 km/h

**Pressure:**
- **Pressure:** 1020 mb
- **Pressure Tendency:** Falling

---

### Rapport Météorologique pour Londres Aujourd'hui

**Date et Heure :** 20 mai 2025, 14h01 UTC+1

**Conditions :**
- **Phrase Météo :** Ensoleillé
- **Précipitations :** Non
- **Couverture Nuageuse :** 10%
- **Visibilité :** 59,5 km
- **Indice UV :** Modéré (5)

**Température :**
- **Tempéra

In [16]:
prompt = "Hello, print the UV informations for London today. Print the date as well of the weather data."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_Rq7m04t1l4SNoc0QLFE7g2sv
Created message ID = msg_QhzNFW7EitJxLMS6PQFbw59y


In [17]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


The UV information for London today, along with the date, is as follows:

- **Date**: May 20, 2025
- **UV Index**: 5
- **UV Index Phrase**: Moderate

The weather conditions for today are also sunny with a temperature of 21.3°C.


In [18]:
prompt = "Hello, What is the temperature in Paris? Display the time of the forecast"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_fafK6aiCEXMgmwSqEvMVTpcS
Created message ID = msg_H9xzGV4hOdewviBdMb4I4qrw


In [19]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


The current temperature in Paris is **24.0°C**. The weather is sunny, and this forecast was recorded at **14:56** local time.


In [20]:
prompt = "Hello, What is the temperature and UV information for Marseille? Display the time of the forecast for the UTC zone"

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_XwX8JMWtshYKaDfKL7sZ9mJf
Created message ID = msg_BGdw2atDhO166KY2SkFU6hRd


In [21]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


The current weather information for Marseille is as follows:

- **Temperature:** 19.5°C
- **UV Index:** 6 (High)
- **Forecast Time (UTC):** The forecast is for **2025-05-20 at 12:56 PM UTC**.

If you need further information, feel free to ask!


In [22]:
prompt = "Generate a weather report for Paris. This report is for a blog publication. Add emojis and adopt a fun style."

# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread ID = {thread.id}")

# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=prompt,
)

print(f"Created message ID = {message.id}")

Created thread ID = thread_N9gR4I1zbB2hFvbOgUmJDR6Z
Created message ID = msg_5XBLD8WTAIGbfHsNeioytyY0


In [23]:
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)

messages = project_client.agents.list_messages(thread_id=thread.id)
print("\033[1;31;34m")
print(messages.data[0].content[0].text.value)


🌞 **Bonjour Paris!** 🌞

Today's weather report for the City of Light is *perfectly magnifique*! The sun is shining bright, and it’s **a stunningly sunny day** with the temperature hovering around a glorious **24°C**! 🏖️

### Here’s what you can expect:
- **🌤 Sky:** Clear blue skies with absolutely no clouds to spoil your view. 
- **💨 Wind:** A pleasant breeze coming in from the NNE at **17.1 km/h**, with occasional gusts hitting **35.4 km/h**. Perfect for a leisurely stroll along the Seine! 
- **☀️ Real Feel:** It seems even warmer, with a real feel of **25.7°C**—so don’t forget your sunny glasses! 😎
- **🌡 Humidity:** A comfy **31%**, meaning it’s just the right amount of crisp in the air!
- **🌈 UV Index:** High at **6**—make sure to slap on some sunscreen before you head out! 🧴

### Wrap-Up:
Feeling warm? You should! With temperatures reaching a delightful **24°C** and lovely conditions to boot, it’s a day made for exploring. Whether you’re picnicking in a park or enjoying a café on 

### Post processing

In [24]:
agents = project_client.agents.list_agents()

for i in range(len(agents.data)):
    if agents.data[i].name == name:
        print(f"Deleting agent {agents.data[i].id}")
        project_client.agents.delete_agent(agents.data[i].id)

Deleting agent asst_6RjTsIQvw7hACHCXHdL7mHs6
